In [1]:

import time
import random
import itertools


## Código de soluciones pasadas

Antes de comenzar con la segunda entrega introduciremos código de las antiguas prácticas para poder disponer de las funciones creadas según necesidad.

In [2]:
#código obtenido a partir de el pseudocódigo descrito en el apartado 3.2.2
#encontrado en: https://datatracker.ietf.org/doc/html/rfc1951#page-7
def CodeCanonico(L):
    #1) Count the number of codes for each code length
    bl_count = [0] * (max(L) + 1)
    for length in L:
        bl_count[length] += 1

    #for i in range(0,len(bl_count)):
    #    print("The value of position "+ str(i) + " is " + str(bl_count[i]))
    
    #2) Find the numerical value of the smallest code for each code length
    next_code = [0] * (max(L) + 1)
    code = 0
    for bits in range(1, max(L) + 1):
        code = (code + bl_count[bits - 1]) << 1
        next_code[bits] = code

    #for i in range(0,len(next_code)):
    #    print("The value of position "+ str(i) + " is " + str(next_code[i]))
    
    #3) Assign numerical values to all codes
    code_canonical = []
    for length in L:
        if length != 0:
            #excluimos los primeros 2 elementos pues son "0b"
            bin_num = bin(next_code[length])[2:]
            #se debe rellenar con zeros pues sino se truncan y los necesitamos
            #pues realmente queremos codificar y que tengan la longitud adecuada
            code_canonical.append(bin_num.zfill(length))
            next_code[length] += 1

    
    return code_canonical

In [3]:
def Encode(M, m2c):
    C = applyDict(M, m2c)
    return C

def Decode(C,c2m):
    M = applyDict(C,c2m)
    return M

#esta funcion es consistente en encode y decode si ningún código es prefijo de otro
def applyDict(A,dict):
    B = ""
    i = 0
    #aquí mantenemos el prefijo que tratamos de sustituir
    prefix = ""
    
    while i<len(A):
        prefix += A[i]
        #print(prefix)
        if prefix in dict:
            B += dict[prefix]
            #vacíamos el prefijo pues ya hemos cambiado el target
            prefix = ""
        i += 1

    #si llegados a este punto el prefijo no se ha vaciado no se ha podido procesar todo el texto
    if prefix != "":
        print("-----------------------------------")
        print("Código no prefijo o se ha encontrado palabra que no es del código: " + prefix)
        print("-----------------------------------")
        return None

    return B

## Nuevas funciones
**Definir una función** `LongitudMedia(C,p)` **que dado un código $C$ y una distribución de probabilidades $p$ devuelva la longitud media $\tilde l$ del código**.

In [4]:
def LongitudMedia(C,p):
    l = 0
    #por si acaso no son probabilidades de 0 a 1 y son números cualesquiera
    divided = [x / sum(p) for x in p]
    for i in range(len(divided)):
        l += len(C[i])*divided[i]
    return l 

**Definir una función** `Huffman(p)` **que dada una distribución de probabilidades $p$ devuelva** ***UN*** **código de Huffman asociado a dicha la distribución de probabilidades**.

In [5]:
import heapq

class BinTreeNode:
    def __init__(self, symbol=None, freq=None, left=None, right=None):
        self.symbol = symbol
        self.freq = freq
        self.left = left
        self.right = right

    #necesario para que se ordenen dentro del heap correctamente
    def __lt__(self, other):
        return self.freq < other.freq
    
def buildTreeOnlyProb(p):
    huff_heap = []

    for i in range(len(p)):
        heapq.heappush(huff_heap,BinTreeNode(i, p[i]))

    while len(huff_heap) > 1:
        left_node = heapq.heappop(huff_heap)
        right_node = heapq.heappop(huff_heap)
        new_node = BinTreeNode(symbol=-1,freq=left_node.freq + right_node.freq, left=left_node, right=right_node)
        heapq.heappush(huff_heap, new_node)

    return huff_heap[0]

def gatherDepth(node, depth, leaf_dict):
    if node is None:
        return
    
    if node.left is None and node.right is None:
        leaf_dict[node.symbol] = depth
    
    gatherDepth(node.left, depth + 1, leaf_dict)
    gatherDepth(node.right, depth + 1, leaf_dict)

def print_tree(node, level=0):
    if node is None:
        return

    print_tree(node.right, level + 1)

    print("\t" * level + str(node.symbol))

    print_tree(node.left, level + 1)

def reconstructLenght(tree):
    length_dic = {}
    gatherDepth(tree, 0, length_dic)
    lenght = [length_dic[i] for i in range(len(length_dic))]
    return lenght

def Huffman(p):
    huff_tree = buildTreeOnlyProb(p)
    #print_tree(huff_tree)
    longitudes_palabras = reconstructLenght(huff_tree)
    return CodeCanonico(longitudes_palabras)

### Ejemplos

In [6]:
p=[0.5,0.1,0.2,0.1,0.05,0.05]
codigo=Huffman(p)
print(p)
print([[p[i],codigo[i]] for i in range(len(p))], '\n\nLongitud Media:',LongitudMedia(codigo,p))

#[0.5, 0.1, 0.2, 0.1, 0.05, 0.05]
#[[0.5, '0'], [0.1, '100'], [0.2, '101'], [0.1, '110'], [0.05, '1110'], [0.05, '1111']] 
#Longitud Media: 2.1

[0.5, 0.1, 0.2, 0.1, 0.05, 0.05]
[[0.5, '0'], [0.1, '100'], [0.2, '101'], [0.1, '110'], [0.05, '1110'], [0.05, '1111']] 

Longitud Media: 2.1


In [7]:
p=[0.8,0.1,0.05,0.05]
codigo=Huffman(p)
print(p)
print([[p[i],codigo[i]] for i in range(len(p))], '\n Longitud Media:', LongitudMedia(codigo,p))

#[0.8, 0.1, 0.05, 0.05]
#[[0.8, '0'], [0.1, '10'], [0.05, '110'], [0.05, '111']] 
#Longitud Media: 1.2999999999999998

[0.8, 0.1, 0.05, 0.05]
[[0.8, '0'], [0.1, '10'], [0.05, '110'], [0.05, '111']] 
 Longitud Media: 1.2999999999999998


In [8]:
p=[0.5,0.25,0.1,0.05,0.05,0.05]
codigo=Huffman(p)
print(p)
print([[p[i],codigo[i]] for i in range(len(p))], '\n Longitud Media:', LongitudMedia(codigo,p))

#[0.5, 0.25, 0.1, 0.05, 0.05, 0.05] 1.0
#[[0.5, '0'], [0.25, '10'], [0.1, '110'], [0.05, '11110'], [0.05, '1110'], [0.05, '11111']] 
#Longitud Media: 2.0

[0.5, 0.25, 0.1, 0.05, 0.05, 0.05]
[[0.5, '0'], [0.25, '10'], [0.1, '1100'], [0.05, '1101'], [0.05, '1110'], [0.05, '1111']] 
 Longitud Media: 1.9999999999999998


In [9]:
p=[0.102, 0.106, 0.053, 0.114, 0.0081, 0.106, 0.0088, 0.030, 0.056, 0.055, 0.032, 0.094, 0.0075, 0.078, 0.1496]
codigo=Huffman(p)
print(p)
print([[p[i],codigo[i]] for i in range(len(p))], '\n Longitud Media:', LongitudMedia(codigo,p))

#[0.102, 0.106, 0.053, 0.114, 0.0081, 0.106, 0.0088, 0.03, 0.056, 0.055, 0.032, 0.094, 0.0075, 0.078, 0.1496] 1.0
#[[0.102, '000'], [0.106, '001'], [0.053, '11100'], [0.114, '010'], [0.0081, '1111110'], [0.106, '011'], [0.0088, '111110'], [0.03, '11101'], [0.056, '1010'], [0.055, '1011'], [0.032, '11110'], [0.094, '1100'], [0.0075, '1111111'], [0.078, '1101'], [0.1496, '100']] 
#Longitud Media: 3.6018000000000003

[0.102, 0.106, 0.053, 0.114, 0.0081, 0.106, 0.0088, 0.03, 0.056, 0.055, 0.032, 0.094, 0.0075, 0.078, 0.1496]
[[0.102, '000'], [0.106, '001'], [0.053, '11100'], [0.114, '010'], [0.0081, '1111110'], [0.106, '011'], [0.0088, '111110'], [0.03, '11101'], [0.056, '1010'], [0.055, '1011'], [0.032, '11110'], [0.094, '1100'], [0.0075, '1111111'], [0.078, '1101'], [0.1496, '100']] 
 Longitud Media: 3.6018000000000003


In [10]:
n=2**7
p=[1/n for _ in range(n)]

codigo=Huffman(p)

print(codigo, '\n Longitud Media:', LongitudMedia(codigo,p))

#['0000000', '0000001', '0000010', '0000011', '0000100', '0000101', '0000110', '0000111', '0001000', '0001001', '0001010', '0001011', '0001100', '0001101', '0001110', '0001111', '0010000', '0010001', '0010010', '0010011', '0010100', '0010101', '0010110', '0010111', '0011000', '0011001', '0011010', '0011011', '0011100', '0011101', '0011110', '0011111', '0100000', '0100001', '0100010', '0100011', '0100100', '0100101', '0100110', '0100111', '0101000', '0101001', '0101010', '0101011', '0101100', '0101101', '0101110', '0101111', '0110000', '0110001', '0110010', '0110011', '0110100', '0110101', '0110110', '0110111', '0111000', '0111001', '0111010', '0111011', '0111100', '0111101', '0111110', '0111111', '1000000', '1000001', '1000010', '1000011', '1000100', '1000101', '1000110', '1000111', '1001000', '1001001', '1001010', '1001011', '1001100', '1001101', '1001110', '1001111', '1010000', '1010001', '1010010', '1010011', '1010100', '1010101', '1010110', '1010111', '1011000', '1011001', '1011010', '1011011', '1011100', '1011101', '1011110', '1011111', '1100000', '1100001', '1100010', '1100011', '1100100', '1100101', '1100110', '1100111', '1101000', '1101001', '1101010', '1101011', '1101100', '1101101', '1101110', '1101111', '1110000', '1110001', '1110010', '1110011', '1110100', '1110101', '1110110', '1110111', '1111000', '1111001', '1111010', '1111011', '1111100', '1111101', '1111110', '1111111'] 
#Longitud Media: 7.0

['0000000', '0000001', '0000010', '0000011', '0000100', '0000101', '0000110', '0000111', '0001000', '0001001', '0001010', '0001011', '0001100', '0001101', '0001110', '0001111', '0010000', '0010001', '0010010', '0010011', '0010100', '0010101', '0010110', '0010111', '0011000', '0011001', '0011010', '0011011', '0011100', '0011101', '0011110', '0011111', '0100000', '0100001', '0100010', '0100011', '0100100', '0100101', '0100110', '0100111', '0101000', '0101001', '0101010', '0101011', '0101100', '0101101', '0101110', '0101111', '0110000', '0110001', '0110010', '0110011', '0110100', '0110101', '0110110', '0110111', '0111000', '0111001', '0111010', '0111011', '0111100', '0111101', '0111110', '0111111', '1000000', '1000001', '1000010', '1000011', '1000100', '1000101', '1000110', '1000111', '1001000', '1001001', '1001010', '1001011', '1001100', '1001101', '1001110', '1001111', '1010000', '1010001', '1010010', '1010011', '1010100', '1010101', '1010110', '1010111', '1011000', '1011001', '1011010'

**Definir una función** `tablaFrecuencias(mensaje, numero_de_simbolos=1)` **que dado un mensaje $M$ devuelva una tabla con la frecuencia de cada tupla de** `numero_de_simbolos` **símbolos del mensaje**.

In [11]:
def tablaFrecuencias(mensaje,numero_de_simbolos=1):
    frecuencias_d = {}
    i = 0
    while i < len(mensaje) - numero_de_simbolos + 1:
        unit = mensaje[i:i + numero_de_simbolos]
        if unit in frecuencias_d:
            frecuencias_d[unit] += 1
        else:
            frecuencias_d[unit] = 1
        i += numero_de_simbolos

    #como el output se da ordenado lexicograficamente para mejor comparación ordenaré las claves del diccionario
    sorted_keys = sorted(frecuencias_d.keys())

    frecuencias = [[symbol, frecuencias_d[symbol]] for symbol in sorted_keys]
    return frecuencias

### Ejemplos de como cambian las frecuencias según la _fuente_ $S$

In [12]:
mensaje='La heroica ciudad dormía la siesta. El viento Sur, caliente y perezoso, empujaba las nubes blanquecinas que se rasgaban al correr hacia el Norte. En las calles no había más ruido que el rumor estridente de los remolinos de polvo, trapos, pajas y papeles que iban de arroyo en arroyo, de acera en acera, de esquina en esquina revolando y persiguiéndose, como mariposas que se buscan y huyen y que el aire envuelve en sus pliegues invisibles. Cual turbas de pilluelos, aquellas migajas de la basura, aquellas sobras de todo se juntaban en un montón, parábanse como dormidas un momento y brincaban de nuevo sobresaltadas, dispersándose, trepando unas por las paredes hasta los cristales temblorosos de los faroles, otras hasta los carteles de papel mal pegado a las esquinas, y había pluma que llegaba a un tercer piso, y arenilla que se incrustaba para días, o para años, en la vidriera de un escaparate, agarrada a un plomo. Vetusta, la muy noble y leal ciudad, corte en lejano siglo, hacía la digestión del cocido y de la olla podrida, y descansaba oyendo entre sueños el monótono y familiar zumbido de la campana de coro, que retumbaba allá en lo alto de la esbeltatorre en la Santa Basílica. La torre de la catedral, poema romántico de piedra,delicado himno, de dulces líneas de belleza muda y perenne, era obra del siglo diez y seis, aunque antes comenzada, de estilo gótico, pero, cabe decir, moderado por un instinto de prudencia y armonía que modificaba las vulgares exageraciones de esta arquitectura. La vista no se fatigaba contemplando horas y horas aquel índice depiedra que señalaba al cielo; no era una de esas torres cuya aguja se quiebra desutil, más flacas que esbeltas, amaneradas, como señoritas cursis que aprietandemasiado el corsé; era maciza sin perder nada de su espiritual grandeza, y hasta sussegundos corredores, elegante balaustrada, subía como fuerte castillo, lanzándosedesde allí en pirámide de ángulo gracioso, inimitable en sus medidas y proporciones.Como haz de músculos y nervios la piedra enroscándose en la piedra trepaba a la altura, haciendo equilibrios de acróbata en el aire; y como prodigio de juegosmalabares, en una punta de caliza se mantenía, cual imantada, una bola grande debronce dorado, y encima otra más pequeña, y sobre ésta una cruz de hierro que acababa en pararrayos.'

nnn=1
a = [[' ', 384], [',', 46], ['.', 8], [';', 3], ['B', 1], ['C', 2], ['E', 2], ['L', 3], ['N', 1], ['S', 2], ['V', 1], ['a', 267], ['b', 46], ['c', 64], ['d', 104], ['e', 238], ['f', 6], ['g', 23], ['h', 15], ['i', 98], ['j', 7], ['l', 112], ['m', 49], ['n', 110], ['o', 141], ['p', 48], ['q', 25], ['r', 126], ['s', 145], ['t', 68], ['u', 87], ['v', 11], ['x', 1], ['y', 29], ['z', 11], ['á', 11], ['é', 3], ['í', 12], ['ñ', 5], ['ó', 5], ['ú', 1]]
b = tablaFrecuencias(mensaje,numero_de_simbolos=nnn)
print(a==b)
print(b)


True
[[' ', 384], [',', 46], ['.', 8], [';', 3], ['B', 1], ['C', 2], ['E', 2], ['L', 3], ['N', 1], ['S', 2], ['V', 1], ['a', 267], ['b', 46], ['c', 64], ['d', 104], ['e', 238], ['f', 6], ['g', 23], ['h', 15], ['i', 98], ['j', 7], ['l', 112], ['m', 49], ['n', 110], ['o', 141], ['p', 48], ['q', 25], ['r', 126], ['s', 145], ['t', 68], ['u', 87], ['v', 11], ['x', 1], ['y', 29], ['z', 11], ['á', 11], ['é', 3], ['í', 12], ['ñ', 5], ['ó', 5], ['ú', 1]]


In [13]:
mensaje='La heroica ciudad dormía la siesta. El viento Sur, caliente y perezoso, empujaba las nubes blanquecinas que se rasgaban al correr hacia el Norte. En las calles no había más ruido que el rumor estridente de los remolinos de polvo, trapos, pajas y papeles que iban de arroyo en arroyo, de acera en acera, de esquina en esquina revolando y persiguiéndose, como mariposas que se buscan y huyen y que el aire envuelve en sus pliegues invisibles. Cual turbas de pilluelos, aquellas migajas de la basura, aquellas sobras de todo se juntaban en un montón, parábanse como dormidas un momento y brincaban de nuevo sobresaltadas, dispersándose, trepando unas por las paredes hasta los cristales temblorosos de los faroles, otras hasta los carteles de papel mal pegado a las esquinas, y había pluma que llegaba a un tercer piso, y arenilla que se incrustaba para días, o para años, en la vidriera de un escaparate, agarrada a un plomo. Vetusta, la muy noble y leal ciudad, corte en lejano siglo, hacía la digestión del cocido y de la olla podrida, y descansaba oyendo entre sueños el monótono y familiar zumbido de la campana de coro, que retumbaba allá en lo alto de la esbeltatorre en la Santa Basílica. La torre de la catedral, poema romántico de piedra,delicado himno, de dulces líneas de belleza muda y perenne, era obra del siglo diez y seis, aunque antes comenzada, de estilo gótico, pero, cabe decir, moderado por un instinto de prudencia y armonía que modificaba las vulgares exageraciones de esta arquitectura. La vista no se fatigaba contemplando horas y horas aquel índice depiedra que señalaba al cielo; no era una de esas torres cuya aguja se quiebra desutil, más flacas que esbeltas, amaneradas, como señoritas cursis que aprietandemasiado el corsé; era maciza sin perder nada de su espiritual grandeza, y hasta sussegundos corredores, elegante balaustrada, subía como fuerte castillo, lanzándosedesde allí en pirámide de ángulo gracioso, inimitable en sus medidas y proporciones.Como haz de músculos y nervios la piedra enroscándose en la piedra trepaba a la altura, haciendo equilibrios de acróbata en el aire; y como prodigio de juegosmalabares, en una punta de caliza se mantenía, cual imantada, una bola grande debronce dorado, y encima otra más pequeña, y sobre ésta una cruz de hierro que acababa en pararrayos.'

nnn=2
a = [[' C', 1], [' L', 1], [' N', 1], [' a', 18], [' b', 5], [' c', 17], [' d', 23], [' e', 21], [' f', 2], [' g', 2], [' h', 5], [' i', 4], [' j', 2], [' l', 15], [' m', 7], [' n', 3], [' o', 4], [' p', 15], [' q', 8], [' r', 5], [' s', 9], [' t', 2], [' u', 5], [' v', 1], [' y', 9], [' á', 1], [' é', 1], [', ', 16], [',d', 1], ['. ', 4], ['..', 1], ['.C', 1], ['; ', 1], ['Ba', 1], ['El', 1], ['En', 1], ['La', 2], ['Sa', 1], ['Su', 1], ['Ve', 1], ['a ', 40], ['a,', 8], ['a.', 1], ['ab', 9], ['ac', 5], ['ad', 7], ['ag', 1], ['ai', 1], ['al', 9], ['am', 1], ['an', 9], ['ap', 2], ['aq', 2], ['ar', 7], ['as', 10], ['at', 3], ['ay', 1], ['añ', 1], ['ba', 14], ['bl', 1], ['br', 3], ['ca', 7], ['ce', 2], ['ci', 7], ['co', 5], ['cr', 3], ['ct', 1], ['cu', 3], ['cí', 1], ['d ', 1], ['d,', 1], ['da', 7], ['de', 23], ['di', 4], ['do', 11], ['dr', 4], ['du', 1], ['e ', 34], ['e,', 3], ['ea', 1], ['eb', 2], ['ed', 3], ['ei', 1], ['el', 11], ['em', 2], ['en', 15], ['ep', 3], ['eq', 1], ['er', 10], ['es', 20], ['et', 1], ['ev', 2], ['ez', 2], ['eñ', 2], ['fa', 1], ['fl', 1], ['fu', 1], ['ga', 7], ['ge', 1], ['gi', 1], ['gl', 1], ['go', 1], ['gr', 2], ['gu', 3], ['ha', 5], ['hi', 2], ['ho', 2], ['hu', 1], ['ia', 3], ['ib', 2], ['ic', 3], ['id', 4], ['ie', 9], ['if', 1], ['ig', 2], ['il', 2], ['im', 1], ['in', 2], ['io', 3], ['ir', 2], ['is', 5], ['it', 1], ['iu', 2], ['iz', 1], ['ié', 1], ['ja', 4], ['l ', 10], ['l,', 1], ['la', 14], ['lc', 1], ['le', 10], ['lg', 1], ['li', 4], ['ll', 6], ['lo', 9], ['lt', 1], ['lu', 1], ['ma', 8], ['mb', 2], ['me', 2], ['mi', 2], ['mn', 1], ['mo', 7], ['mp', 2], ['mu', 1], ['má', 2], ['n ', 14], ['na', 6], ['nc', 4], ['nd', 8], ['ne', 2], ['ng', 1], ['ni', 1], ['nn', 1], ['no', 6], ['nq', 1], ['ns', 2], ['nt', 8], ['nu', 1], ['nv', 1], ['nz', 1], ['nó', 1], ['o ', 20], ['o,', 6], ['o;', 1], ['ob', 2], ['oc', 1], ['od', 4], ['oe', 1], ['oi', 1], ['ol', 5], ['om', 6], ['on', 4], ['op', 1], ['or', 8], ['os', 8], ['ot', 1], ['pa', 7], ['pe', 8], ['pi', 3], ['pl', 1], ['po', 3], ['pr', 2], ['pu', 1], ['qu', 10], ['r ', 5], ['r,', 2], ['ra', 18], ['rb', 1], ['rc', 1], ['rd', 1], ['re', 11], ['ri', 5], ['rm', 2], ['ro', 7], ['rq', 1], ['rr', 4], ['rs', 3], ['rt', 1], ['ru', 3], ['rv', 1], ['rá', 1], ['s ', 34], ['s,', 8], ['s.', 1], ['sa', 3], ['sb', 2], ['sc', 3], ['se', 10], ['si', 4], ['sm', 1], ['so', 4], ['sq', 2], ['st', 6], ['su', 4], ['sí', 1], ['ta', 11], ['te', 9], ['ti', 6], ['to', 5], ['tr', 7], ['tu', 3], ['u ', 1], ['ua', 1], ['ub', 2], ['ud', 1], ['ue', 12], ['ui', 6], ['uj', 1], ['ul', 1], ['um', 1], ['un', 9], ['ur', 2], ['us', 3], ['ut', 1], ['uy', 1], ['uz', 1], ['ve', 1], ['vi', 2], ['vo', 1], ['vu', 2], ['xa', 1], ['y ', 14], ['ye', 2], ['yo', 2], ['z ', 1], ['za', 2], ['zo', 1], ['zu', 1], ['zá', 1], ['á ', 1], ['ám', 1], ['án', 3], ['ás', 1], ['é;', 1], ['í ', 1], ['ía', 7], ['ín', 2], ['ña', 1], ['ño', 1], ['ób', 1], ['ón', 2], ['ót', 1], ['ús', 1]]
b = tablaFrecuencias(mensaje,numero_de_simbolos=nnn)
print(a==b)
print(b)

False
[[' C', 1], [' L', 1], [' N', 1], [' a', 18], [' b', 5], [' c', 17], [' d', 23], [' e', 21], [' f', 2], [' g', 2], [' h', 5], [' i', 4], [' j', 2], [' l', 15], [' m', 7], [' n', 3], [' o', 4], [' p', 15], [' q', 8], [' r', 5], [' s', 9], [' t', 2], [' u', 5], [' v', 1], [' y', 9], [' á', 1], [' é', 1], [', ', 16], [',d', 1], ['. ', 4], ['.C', 1], ['; ', 1], ['Ba', 1], ['El', 1], ['En', 1], ['La', 2], ['Sa', 1], ['Su', 1], ['Ve', 1], ['a ', 40], ['a,', 8], ['a.', 1], ['ab', 9], ['ac', 5], ['ad', 7], ['ag', 1], ['ai', 1], ['al', 9], ['am', 1], ['an', 9], ['ap', 2], ['aq', 2], ['ar', 7], ['as', 10], ['at', 3], ['ay', 1], ['añ', 1], ['ba', 14], ['bl', 1], ['br', 3], ['ca', 7], ['ce', 2], ['ci', 7], ['co', 5], ['cr', 3], ['ct', 1], ['cu', 3], ['cí', 1], ['d ', 1], ['d,', 1], ['da', 7], ['de', 23], ['di', 4], ['do', 11], ['dr', 4], ['du', 1], ['e ', 34], ['e,', 3], ['ea', 1], ['eb', 2], ['ed', 3], ['ei', 1], ['el', 11], ['em', 2], ['en', 15], ['ep', 3], ['eq', 1], ['er', 10], ['es', 20

---
**Definir una función** `EncodeHuffman(mensaje,numero_de_simbolos=n_simbolos)` **que codifique un mensaje utilizando un código de Huffman 
obtenido a partir de las frecuencias de los caracteres del mensaje.**


In [14]:
def EncodeHuffman(mensaje,numero_de_simbolos=1):
    symbol_n_freq = tablaFrecuencias(mensaje,numero_de_simbolos=numero_de_simbolos)
    freq = [symbol_n_freq[i][1] for i in range(len(symbol_n_freq))]
    huff_code = Huffman(freq)
    m2c = dict([(symbol_n_freq[i][0],huff_code[i])for i in range(len(freq))])
    encoded_message = Encode(mensaje,m2c)
    mean_length = LongitudMedia(huff_code,freq)
    #print([[symbol_n_freq[i][0],freq[i],huff_code[i]] for i in range(len(freq))])
    #print(symbol_n_freq)
    #print(freq)
    #print(m2c)
    #print(LongitudMedia(huff_code,freq))

    return encoded_message, m2c, mean_length

In [15]:
def DecodeHuffman(coded, m2c, length):
    c2m = dict([(c,m) for m, c in m2c.items()])
    mensaje_reconstruido = Decode(coded,c2m)
    return mensaje_reconstruido

## Código para el cuestionario

A partir de aquí el código no se corresponde al necesario para responder al cuestionario así que desarrollare mis propias funciones


In [16]:
#Pregunta 1

prob = [ 23/73 , 20/73 , 14/73 , 13/73 , 3/73 ]

code = Huffman(prob)

print(LongitudMedia(code,prob))

2.2191780821917813


In [17]:
#Pregunta 2

mensaje="The Invisible Man, by H. G. Wells. [...] I stood perfectly still while he stared about open-mouthed and suspicious. 'It must have been her,' he said slowly. 'Damn her!' He shut the door quietly, and immediately I heard the key turn in the lock. Then his footsteps retreated. I realised abruptly that I was locked in. For a minute I did not know what to do. I walked from door to window and back, and stood perplexed. A gust of anger came upon me. But I decided to inspect the clothes before I did anything further, and my first attempt brought down a pile from an upper shelf. This brought him back, more sinister than ever. That time he actually touched me, jumped back with amazement and stood astonished in the middle of the room."

nnn=1

mensaje_codificado, m2c,len_M = EncodeHuffman(mensaje,nnn)

print(len_M)

4.452933151432472


In [18]:
#Pregunta 3

#multirespuesta adivina el código de huffman

a = [0.0333, 0.0099, 0.0327, 0.0031, 0.0254, 0.0253, 0.0045, 0.0237, 0.0181, 0.7061, 0.033, 0.0045, 0.0354, 0.018, 0.0271]
code_a = Huffman(a)
canon_a = [len(elem) for elem in code_a]

b = [0.0014, 0.0204, 0.0356, 0.0475, 0.0517, 0.0518, 0.0591, 0.0591, 0.0708, 0.08, 0.0811, 0.0909, 0.1064, 0.1152, 0.1292]
code_b = Huffman(b)
canon_b = [len(elem) for elem in code_b]

c = [0.0591, 0.08, 0.0518, 0.0014, 0.1152, 0.0909, 0.0708, 0.0517, 0.1064, 0.1292, 0.0811, 0.0591, 0.0475, 0.0204, 0.0356]
code_c = Huffman(c)
canon_c = [len(elem) for elem in code_c]

test =  ['1111', '1001', '0111', '011000', '010', '1010', '1000', '0001', '001', '110', '1011', '1110', '0000', '011001', '01101']
code_test = [len(elem) for elem in test]

if(canon_a == code_test):
    print("code a is the right response")

elif(canon_b == code_test):
    print("code b is the right response")

elif(canon_c == code_test):
    print("code c is the right response")

else:
    print("something went wrong")

code c is the right response


In [19]:
#Pregunta 4

mensaje="The Invisible Man, by H. G. Wells. [...] Dr. Kemp was a tall and slender young man, with flaxen hair and a moustache almost white, and the work he was upon would earn him, he hoped, the fellowship of the Royal Society, so highly did he think of it. And his eye, presently wandering from his work, caught the sunset blazing at the back of the hill that is over against his own. For a minute perhaps he sat, pen in mouth, admiring the rich golden colour above the crest, and then his attention was attracted by the little figure of a man, inky black, running over the hill-brow towards him. He was a shortish little man, and he wore a high hat, and he was running so fast that his legs verily twinkled."

nnn=1

mensaje_codificado, m2c,len_M = EncodeHuffman(mensaje,nnn)

print(len_M)

4.35


In [20]:
#Pregunta 5

mensaje="The Island of Doctor Moreau, by H. G. Wells. [...] I found myself that the cries were singularly irritating, and they grew in depth and intensity as the afternoon wore on. They were painful at first, but their constant resurgence at last altogether upset my balance. I flung aside a crib of Horace I had been reading, and began to clench my fists, to bite my lips, and to pace the room. Presently I got to stopping my ears with my fingers. The emotional appeal of those yells grew upon me steadily, grew at last to such an exquisite expression of suffering that I could stand it in that confined room no longer. I stepped out of the door into the slumberous heat of the late afternoon, and walking past the main entrance--locked again, I noticed--turned the corner of the wall."

nnn=1

symbol_n_freq = tablaFrecuencias(mensaje,numero_de_simbolos=nnn)
freq = [symbol_n_freq[i][1] for i in range(len(symbol_n_freq))]
huff_code = Huffman(freq)

a = ['00', '110010', '1111000', '110011', '111111100', '11110110', '11110111', '11111000', '11111001', '11111010', '11111011', '1111001', '111111101', '111111110', '0100', '1111010', '110100', '110101', '0101', '110110', '10100', '10101', '10110', '11111100', '10111', '110111', '0110', '0111', '111000', '11111101', '1000', '11000', '1001', '111001', '111010', '111111111', '111011']

b = ['00', '1110110', '11111000', '110100', '1110111', '11111001', '11111010', '110101', '111111100', '1111000', '11111011', '111111101', '11111100', '1111001', '0100', '1111010', '10010', '10011', '0101', '10100', '110110', '10101', '10110', '111111110', '10111', '110111', '0110', '0111', '111000', '111111111', '11000', '11001', '1000', '111001', '1111011', '11111101', '111010']

c = ['000', '110110', '11111010', '110111', '1111111000', '1111111001', '11111011', '1111010', '1111111010', '1111111011', '11111100', '1111111100', '1111111101', '1111111110', '0100', '1111011', '111000', '10110', '001', '10111', '11000', '11001', '0101', '111111010', '11010', '111001', '0110', '0111', '111010', '1111111111', '1000', '1001', '1010', '111011', '1111100', '111111011', '111100']

if(LongitudMedia(a,freq) == LongitudMedia(huff_code,freq)):
    print("code a is the right response")

elif(LongitudMedia(b,freq) == LongitudMedia(huff_code,freq)):
    print("code b is the right response")

elif(LongitudMedia(c,freq) == LongitudMedia(huff_code,freq)):
    print("code c is the right response")

else:
    print("something went wrong")

code c is the right response


Codifico un mensaje de prueba tomando las frecuencias de cada símbolo.  

Hago una estimación de la ratio de compresión sin contar que he de almacenar el diccionario y contándolo.

Los tiempos dependerán del entorno de ejecución.

In [66]:

mensaje='La heroica ciudad dormía la siesta. El viento Sur, caliente y perezoso, empujaba las nubes blanquecinas que se rasgaban al correr hacia el Norte. En las calles no había más ruido que el rumor estridente de los remolinos de polvo, trapos, pajas y papeles que iban de arroyo en arroyo, de acera en acera, de esquina en esquina revolando y persiguiéndose, como mariposas que se buscan y huyen y que el aire envuelve en sus pliegues invisibles. Cual turbas de pilluelos, aquellas migajas de la basura, aquellas sobras de todo se juntaban en un montón, parábanse como dormidas un momento y brincaban de nuevo sobresaltadas, dispersándose, trepando unas por las paredes hasta los cristales temblorosos de los faroles, otras hasta los carteles de papel mal pegado a las esquinas, y había pluma que llegaba a un tercer piso, y arenilla que se incrustaba para días, o para años, en la vidriera de un escaparate, agarrada a un plomo. Vetusta, la muy noble y leal ciudad, corte en lejano siglo, hacía la digestión del cocido y de la olla podrida, y descansaba oyendo entre sueños el monótono y familiar zumbido de la campana de coro, que retumbaba allá en lo alto de la esbeltatorre en la Santa Basílica. La torre de la catedral, poema romántico de piedra,delicado himno, de dulces líneas de belleza muda y perenne, era obra del siglo diez y seis, aunque antes comenzada, de estilo gótico, pero, cabe decir, moderado por un instinto de prudencia y armonía que modificaba las vulgares exageraciones de esta arquitectura. La vista no se fatigaba contemplando horas y horas aquel índice depiedra que señalaba al cielo; no era una de esas torres cuya aguja se quiebra desutil, más flacas que esbeltas, amaneradas, como señoritas cursis que aprietandemasiado el corsé; era maciza sin perder nada de su espiritual grandeza, y hasta sussegundos corredores, elegante balaustrada, subía como fuerte castillo, lanzándosedesde allí en pirámide de ángulo gracioso, inimitable en sus medidas y proporciones.Como haz de músculos y nervios la piedra enroscándose en la piedra trepaba a la altura, haciendo equilibrios de acróbata en el aire; y como prodigio de juegosmalabares, en una punta de caliza se mantenía, cual imantada, una bola grande debronce dorado, y encima otra más pequeña, y sobre ésta una cruz de hierro que acababa en pararrayos.'

mensaje=mensaje[:]*100
n_simbolos=1

mensaje_codificado, m2c,len_M=EncodeHuffman(mensaje,numero_de_simbolos=n_simbolos)
mensaje_recuperado=DecodeHuffman(mensaje_codificado,m2c,len_M)

print('\n Diccionario ordenado alfabéticamente: ', sorted(m2c.items(), key=lambda x: x[0]))
print('\n Diccionario ordenado longitud código: ', sorted(m2c.items(), key=lambda x: len(x[1])))
ratio_compresion=8*len(mensaje)/len(mensaje_codificado)
ratio_compresion_con_diccionario=8*len(mensaje)/(len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1))
print('\n Ratio de compresión: ', ratio_compresion ,8*len(mensaje),"-->",len(mensaje_codificado))
print('\n Ratio de compresión estimada con diccionario: ', ratio_compresion_con_diccionario,8*len(mensaje),"-->",len(mensaje_codificado)+len(m2c)*16)
if (mensaje!=mensaje_recuperado):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

print('\n Diccionario a guardar: ',[[x[0],len(x[1])] for x in m2c.items() ])


------------------------------------------------------------
Tiempo para crear tabla de frecuencias:  0.11396598815917969
Entropía por símbolo:  4.201875546880906
Ratio de compresión máxima estimada sin diccionario:  1.9039116962753644
Tiempo para crear código Huffman:  0.000690460205078125
Tiempo para codificar:  0.033026933670043945

 Diccionario ordenado alfabéticamente:  [(' ', '110'), (',', '111001'), ('.', '10110000'), (';', '000010001'), ('B', '0000000100'), ('C', '1011000101'), ('E', '1011000100'), ('L', '000010000'), ('N', '10110001111'), ('S', '0000000101'), ('V', '10110001110'), ('a', '011'), ('b', '111000'), ('c', '10000'), ('d', '11111'), ('e', '010'), ('f', '00000000'), ('g', '1011011'), ('h', '0000101'), ('i', '11110'), ('j', '00001001'), ('l', '0010'), ('m', '111011'), ('n', '0001'), ('o', '1001'), ('p', '111010'), ('q', '000001'), ('r', '0011'), ('s', '1010'), ('t', '10001'), ('u', '10111'), ('v', '10110101'), ('x', '10110001101'), ('y', '000011'), ('z', '10110100'), (

### Codifico _La Regenta_ 
Uso como fuente los símbolos del texto.

In [71]:
n_simbolos=1

with open ("../standard_test_text/la_regenta_utf8", "r") as myfile:
    mensaje=myfile.read()
t0=time.time()    
mensaje_codificado, m2c,len_M=EncodeHuffman(mensaje,numero_de_simbolos=n_simbolos)
t_encode=time.time()-t0
print('--------------------------------------------------------------------------------')
print('\n Diccionario ordenado alfabéticamente: ', sorted(m2c.items(), key=lambda x: x[0]))
print('\n Diccionario ordenado longitud código: ', sorted(m2c.items(), key=lambda x: len(x[1])))
ratio_compresion=8*len(mensaje)/len(mensaje_codificado)
ratio_compresion_con_diccionario=8*len(mensaje)/(len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1))
print('\n Ratio de compresión: ', ratio_compresion ,8*len(mensaje),"-->",len(mensaje_codificado),'(bits)')
print('\n Ratio de compresión estimada con diccionario: ', ratio_compresion_con_diccionario,8*len(mensaje),"-->",len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1),'(bits)')

print('\n Tamaño Huffman     :',int((len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1))/8),'bytes')
print('\n Tamaño Fichero .bz2: 518588 bytes')
print('\n Tamaño Fichero .zip: 678394 bytes')
print('\n Tamaño Fichero  .gz: 710464 bytes')


t0=time.time()
mensaje_recuperado=DecodeHuffman(mensaje_codificado,m2c,len_M)
t_decode=time.time()-t0
print("\n Tiempo Encode (Tabla frecuencias+Código Huffman+Codificación):", t_encode)
print("\n Tiempo Decode:", t_decode)
if (mensaje!=mensaje_recuperado):
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')



Tiempo para crear tabla de frecuencias:  0.744138240814209
Entropía por símbolo:  4.421453902139422
Ratio de compresión máxima estimada sin diccionario:  1.8093595855718445
Tiempo para crear código Huffman:  0.001974344253540039
Tiempo para codificar:  0.17607498168945312
--------------------------------------------------------------------------------

 Diccionario ordenado alfabéticamente:  [('\n', '110000'), (' ', '111'), ('!', '1011001100'), ("'", '10010000001100101'), ('(', '10110011111110'), (')', '10110011111101'), ('+', '10110011111111000111'), (',', '011101'), ('-', '110101010'), ('.', '100000'), ('0', '10110011111111000101'), ('1', '101100111111111011'), ('2', '101100111111110000'), ('3', '1001000000110011011'), ('4', '10010000001100110000'), ('5', '101100111111110001101'), ('6', '1011001111111100101'), ('7', '1001000000110011010'), ('8', '100100000011001111'), (':', '10010111101'), (';', '100101110'), ('?', '1001000111'), ('A', '011100000'), ('B', '101100111110'), ('C', '1001

**Hago distintas pruebas  para ver como varía la ratio de compresión según el número de símbolos de la fuente tomados para calcular las frecuencias**

In [79]:
with open ("../standard_test_text/la_regenta_utf8", "r") as myfile:
    mensaje=myfile.read()


for n_simbolos in range(1,5):
    
    print('\n\n--------------------------------------------------------------------------------')
    print(' Codifico usando una fuente que toma '+str(n_simbolos)+ ' símbolos consecutivos')
    print('--------------------------------------------------------------------------------')

    t0=time.time()    
    mensaje_codificado, m2c,len_M=EncodeHuffman(mensaje,numero_de_simbolos=n_simbolos)
    t_encode=time.time()-t0
    ratio_compresion=8*len(mensaje)/len(mensaje_codificado)
    ratio_compresion_con_diccionario=8*len(mensaje)/(len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1))
    print('\n Ratio de compresión: ', ratio_compresion ,8*len(mensaje),"-->",len(mensaje_codificado),'(bits)')
    print('\n Ratio de compresión estimada con diccionario: ', ratio_compresion_con_diccionario,8*len(mensaje),"-->",len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1),'(bits)')

    print('\n Tamaño Huffman     :',int((len(mensaje_codificado)+len(m2c)*8*(n_simbolos+1))/8),'bytes')
    print('\n Tamaño Fichero .bz2: 518588 bytes')
    print('\n Tamaño Fichero .zip: 678394 bytes')
    print('\n Tamaño Fichero  .gz: 710464 bytes')

    t0=time.time()
    mensaje_recuperado=DecodeHuffman(mensaje_codificado,m2c,len_M)
    t_decode=time.time()-t0
    print("\n Tiempo Encode (Tabla frecuencias+Código Huffman+Codificación):", t_encode)
    print("\n Tiempo Decode:", t_decode)
    if (mensaje!=mensaje_recuperado):
            print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')





--------------------------------------------------------------------------------
 Codifico usando una fuente que toma 1 símbolos consecutivos
--------------------------------------------------------------------------------
Tiempo para crear tabla de frecuencias:  0.6799380779266357
Entropía por símbolo:  4.421453902139422
Ratio de compresión máxima estimada sin diccionario:  1.8093595855718445
Tiempo para crear código Huffman:  0.0017235279083251953
Tiempo para codificar:  0.1780836582183838

 Ratio de compresión:  1.793636939741042 14344224 --> 7997284 (bits)

 Ratio de compresión estimada con diccionario:  1.7932889235383749 14344224 --> 7998836 (bits)

 Tamaño Huffman     : 999854 bytes

 Tamaño Fichero .bz2: 518588 bytes

 Tamaño Fichero .zip: 678394 bytes

 Tamaño Fichero  .gz: 710464 bytes

 Tiempo Encode (Tabla frecuencias+Código Huffman+Codificación): 0.8636305332183838

 Tiempo Decode: 0.8202612400054932


---------------------------------------------------------------------